In [255]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import pandas as pd
import ijson
import numpy as np
import time

#options = Options()
#options.binary_location = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
chromedriver='C:/Users/default.DESKTOP-FVT3076/Desktop/crawl/chromedriver.exe'
#chromedriver = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
driver = webdriver.Chrome(chromedriver)
time.sleep(1)


In [260]:
# 파일 읽기
# f = open(f"./data/data.json", encoding="UTF-8")
# objects = ijson.items(f, 'item')
# df = pd.DataFrame(objects)

df = pd.read_csv('./2021-09-18_test_실패.csv', sep=',', encoding='CP949')

# df = df[['업소명', '소재지도로명', '업태명', '주된음식', '행정동명', '소재지전화번호']]
# df.columns = ['name', 'address', 'cate1', 'cate2', 'dong', 'phone']
# df = df.drop_duplicates(['name'], keep='first')
# # 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.
# df['cate_mix'] = df['cate1'] + df['cate2']

print(df)

       Unnamed: 0      id           name branch  area            tel  \
0               2       3  Battered Sole    NaN   이태원    02-749-6867   
1               3       4      Chakyoung    NaN  달맞이길   051-756-5566   
2               5       6     Gogoss Pub    NaN    하남   031-792-9289   
3               6       7      Madeleine    NaN   만촌동   053-759-7947   
4               9      10             T원    NaN    신촌   02-3145-2904   
...           ...     ...            ...    ...   ...            ...   
59757      459983  459996         행복부대찌개    NaN   녹양동   031-877-2410   
59758      459984  459997           행복부페    NaN   안중읍   031-682-1733   
59759      459985  459998           행복분식    NaN   퇴촌면  031-8071-2668   
59760      459986  459999           행복분식    NaN   소주동   055-312-3713   
59761      459987  460000           행복분식    NaN  의정부동   031-772-6624   

                     address   latitude   longitude  \
0       서울특별시 용산구 이태원동 118-9  37.535032  126.991664   
1      부산광역시 해운대구 중2동 150

In [182]:
keyword = "산정호수%20해룡식당"
try:
    naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"

    driver.get(naver_map_search_url)
#         time.sleep(0.5)
    x= driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
    # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
    # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.

    #만약 검색 결과가 없다면?
except Exception as e1:
    if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
        print(e1)
        try:
            x = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
#                 time.sleep(0.5)print(x)
        except Exception as e2:
                        if i < 10:
                            print(e2)
                        x = np.nan
        #                 time.sleep(0.5)
    else:
        pass
print(x)

18153025


In [261]:
# df['naver_keyword'] = df['area'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.

for i in range(0,df.shape[0]):
    if df.iloc[i]['area'] is None or df.iloc[i,-2] != df.iloc[i,-2]: #nan 찾는 방법
        df.iloc[i,-2] = df.iloc[i]['name']
#     print(df.iloc[i,-2])
    df['naver_keyword'].iloc[i] = df['naver_keyword'].iloc[i].replace('&amp;','')


C:\Users\default.DESKTOP-FVT3076\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [165]:
df = df.loc[~df['name'].isnull()]

In [262]:
# df.at[90,'naver_keyword'] = df.iloc[90]['name']
df.at[90,'naver_keyword']

'1010포르테스'

In [ ]:
df[(df['area'] is None)].index

In [263]:
df.iloc[25258]

Unnamed: 0                                      425484
id                                              425497
name                                             파리바게뜨
branch                                             NaN
area                                               평거동
tel                                       055-745-8298
address                            경상남도 진주시 평거동 300-10
latitude                                        35.172
longitude                                      128.059
category_list                                       []
menu_list                                           []
bhour_list                                          []
review_cnt                                           0
review_list                                         []
naver_keyword                              평거동%20파리바게뜨
naver_map_url    https://m.place.naver.com/restaurant/
Name: 25258, dtype: object

In [228]:
# df.isnull().sum()
df['area'].isnull().index
# df.shape

RangeIndex(start=0, stop=59763, step=1)

In [264]:
df
#59763행

,Unnamed: 0,id,name,branch,area,tel,address,latitude,longitude,category_list,menu_list,bhour_list,review_cnt,review_list,naver_keyword,naver_map_url
0,2,3,Battered Sole,NaN,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,"[{'category': '피쉬앤칩스'}, {'category': '펍'}]","[{'menu': '메인 (피쉬앤칩스, 오리 가슴살 등)', 'price': 140...","[{'type': 1, 'week_type': 1, 'mon': 0, 'tue': ...",0,[],이태원%20Battered Sole,https://m.place.naver.com/restaurant/35162395
1,3,4,Chakyoung,NaN,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,"[{'category': '레스토랑'}, {'category': '카프레제'}]",[],[],0,[],달맞이길%20Chakyoung,https://m.place.naver.com/restaurant/1128117709
2,5,6,Gogoss Pub,NaN,하남,031-792-9289,경기도 하남시 덕풍동 412-18,37.539046,127.203933,[{'category': '펍'}],"[{'menu': '크리스피후라이드치킨', 'price': 16000}]",[],0,[],하남%20Gogoss Pub,https://m.place.naver.com/restaurant/34582077
3,6,7,Madeleine,NaN,만촌동,053-759-7947,대구광역시 수성구 만촌1동 1356-7,35.869088,128.636058,"[{'category': '마들렌'}, {'category': '베이커리'}]","[{'menu': '레몬핫티', 'price': 3500}, {'menu': '아메...","[{'type': 1, 'week_type': 1, 'mon': 1, 'tue': ...",0,[],만촌동%20Madeleine,https://m.place.naver.com/restaurant/33742254
4,9,10,T원,NaN,신촌,02-3145-2904,서울특별시 서대문구 창천동 30-33,37.556074,126.935798,"[{'category': '중식당'}, {'category': '레스토랑'}]",[],[],0,[],신촌%20T원,https://m.place.naver.com/restaurant/1764176165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,459983,459996,행복부대찌개,NaN,녹양동,031-877-2410,경기도 의정부시 녹양동 154-6,37.757939,127.032758,[],[],[],0,[],녹양동%20행복부대찌개,https://m.place.naver.com/restaurant/
59758,459984,459997,행복부페,NaN,안중읍,031-682-1733,경기도 평택시 안중읍 용성리 718-3,37.002743,126.944991,[],[],[],0,[],안중읍%20행복부페,https://m.place.naver.com/restaurant/
59759,459985,459998,행복분식,NaN,퇴촌면,031-8071-2668,경기도 광주 퇴촌면 도수리 611-6,37.471677,127.315744,[],[],[],0,[],퇴촌면%20행복분식,https://m.place.naver.com/restaurant/
59760,459986,459999,행복분식,NaN,소주동,055-312-3713,경상남도 양산시 소주동 147-1,35.407860,129.158564,[],[],[],0,[],소주동%20행복분식,https://m.place.naver.com/restaurant/


In [266]:
df['naver_keyword'].value_counts()

연신내%20피자알볼로       1
미아사거리%20한옥쌈밥      1
청주%20학천면옥         1
김포%20하오란양꼬치       1
기장%20퀸즈브라운        1
                 ..
성남동%20토리포크        1
동해%20풍어회도매센터      1
신월동%20큰손참숯생불고기    1
서면%20해물장터         1
동대문%20파파존스        1
Name: naver_keyword, Length: 55391, dtype: int64

In [265]:
df = df.drop_duplicates(['naver_keyword'], keep='first') # 4371 사라짐
59763-55392 

4371

In [ ]:
%%time


# 본격적으로 가게 상세페이지의 URL을 가져옵시다
for i, keyword in enumerate(tqdm(df['naver_keyword'].tolist())):
    if i < 10:
        print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
# for i in crawl_range:
#     keyword = df['naver_keyword'][i]
#     print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)

#     if(len(df['naver_map_url'].iloc[0]) > 37):
#         print(df['naver_map_url'].iloc[0] + " PASS")
#         continue
    
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(0.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(0.5)
            except Exception as e2:
                if i < 10:
                    print(e2)
                df.iloc[i,-1] = np.nan
                print("못 찾은 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
                time.sleep(0.5)
        else:
            pass


driver.quit() 


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

이번에 찾을 키워드 : 0 / 55390 행 이태원%20Battered Sole
이번에 찾을 키워드 : 1 / 55390 행 달맞이길%20Chakyoung
이번에 찾을 키워드 : 2 / 55390 행 하남%20Gogoss Pub
이번에 찾을 키워드 : 3 / 55390 행 만촌동%20Madeleine
이번에 찾을 키워드 : 4 / 55390 행 신촌%20T원
이번에 찾을 키워드 : 5 / 55390 행 월곶%20국수이야기
이번에 찾을 키워드 : 6 / 55390 행 역삼역%20깐부치킨
이번에 찾을 키워드 : 7 / 55390 행 수영역%20모이바바
이번에 찾을 키워드 : 8 / 55390 행 유성%20미랑
이번에 찾을 키워드 : 9 / 55390 행 전포동%20써리힐


In [258]:
df

,Unnamed: 0,id,name,branch,area,tel,address,latitude,longitude,category_list,...,bhour_list,review_cnt,review_list,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty
0,2,3,Battered Sole,NaN,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,"[{'category': '피쉬앤칩스'}, {'category': '펍'}]",...,"[{'type': 1, 'week_type': 1, 'mon': 0, 'tue': ...",0,[],이태원%20Battered Sole,https://m.place.naver.com/restaurant/35162395,null,null,null,null,null
1,3,4,Chakyoung,NaN,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,"[{'category': '레스토랑'}, {'category': '카프레제'}]",...,[],0,[],달맞이길%20Chakyoung,https://m.place.naver.com/restaurant/1128117709,양식,4.41,16,차경에 방문해 보신 최근 고객님들은 메뉴에서 'Sweet Cucumber Pickl...,61
2,5,6,Gogoss Pub,NaN,하남,031-792-9289,경기도 하남시 덕풍동 412-18,37.539046,127.203933,[{'category': '펍'}],...,[],0,[],하남%20Gogoss Pub,https://m.place.naver.com/restaurant/34582077,"맥주,호프",4.05,11,이번주는 허군이 월요일부터 허리가 계속 아파서 병원갔다가 바로 퇴근하느라고 퇴근이 ...,3
3,6,7,Madeleine,NaN,만촌동,053-759-7947,대구광역시 수성구 만촌1동 1356-7,35.869088,128.636058,"[{'category': '마들렌'}, {'category': '베이커리'}]",...,"[{'type': 1, 'week_type': 1, 'mon': 1, 'tue': ...",0,[],만촌동%20Madeleine,https://m.place.naver.com/restaurant/33742254,베이커리,4.26,129,SK 리더스뷰 근처 마들렌 다녀왔어요~ㅋㅋ 빵먹을 생각은 없었는데 동네마실겸 식빵사...,18
4,9,10,T원,NaN,신촌,02-3145-2904,서울특별시 서대문구 창천동 30-33,37.556074,126.935798,"[{'category': '중식당'}, {'category': '레스토랑'}]",...,[],0,[],신촌%20T원,https://m.place.naver.com/restaurant/1764176165,null,null,null,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,305715,305728,우돈한마당,NaN,용이동,031-654-2446,경기도 평택시 용이동 600-1,36.998265,127.132317,[],...,[],0,[],용이동%20우돈한마당,https://m.place.naver.com/restaurant/1291607078,null,null,null,null,null
682,328034,328047,이장군족발,NaN,의정부,031-848-4882,경기도 의정부시 금오동 384-9,37.748524,127.053600,"[{'category': '족발'}, {'category': '보쌈'}]",...,[],0,[],의정부%20이장군족발,https://m.place.naver.com/restaurant/37425448,"족발,보쌈",4.39,173,"전국을 다니면서 만나는 사람들 , 유적지 여행 그리고 또 하나의 기쁨 먹거리. 기대...",11
683,350012,350025,제일장수촌,NaN,포승읍,031-682-1880,경기도 평택시 포승읍 석정리 1116-9,36.986675,126.890730,[],...,[],0,[],포승읍%20제일장수촌,https://m.place.naver.com/restaurant/1788716626,null,null,null,null,null
684,352003,352016,조개야,NaN,동래,010-4909-3676,부산광역시 동래구 명륜동 548-42,35.205555,129.079422,"[{'category': '조개구이'}, {'category': '모듬조개'}]",...,"[{'type': 1, 'week_type': 1, 'mon': 1, 'tue': ...",0,[],동래%20조개야,https://m.place.naver.com/restaurant/1522795941,null,null,null,null,null


In [256]:
naver_map_type_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

for i, url in enumerate(tqdm(df['naver_map_url'])):
# for i in tqdm(crawl_range):
#     url = df['naver_map_url'][i]
    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(1)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언

        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_elements_by_class_name("_2CbII")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element_by_tag_name('div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text)

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')


        
driver.quit()
sub_driver.quit()


df['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
df['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수
df['naver_blog_review_txt'] = blog_review_list  # 네이버 상세페이지에 나온 블로그 리뷰 텍스트들
df['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

0행 문제가 발생
4행 문제가 발생
5행 문제가 발생
7행 문제가 발생
8행 문제가 발생
11행 문제가 발생
12행 문제가 발생
13행 문제가 발생
14행 문제가 발생
15행 문제가 발생
16행 문제가 발생
17행 문제가 발생
18행 문제가 발생
21행 문제가 발생
24행 문제가 발생
29행 문제가 발생
30행 문제가 발생
33행 문제가 발생
34행 문제가 발생
38행 문제가 발생
39행 문제가 발생
44행 문제가 발생
49행 문제가 발생
50행 문제가 발생
51행 문제가 발생
52행 문제가 발생
53행 문제가 발생
58행 문제가 발생
59행 문제가 발생
65행 문제가 발생
80행 문제가 발생
87행 문제가 발생
90행 문제가 발생
93행 문제가 발생
103행 문제가 발생
104행 문제가 발생
105행 문제가 발생
106행 문제가 발생
108행 문제가 발생
113행 문제가 발생
116행 문제가 발생
121행 문제가 발생
123행 문제가 발생
124행 문제가 발생
125행 문제가 발생
131행 문제가 발생
134행 문제가 발생
137행 문제가 발생
141행 문제가 발생
145행 문제가 발생
146행 문제가 발생
148행 문제가 발생
152행 문제가 발생
156행 문제가 발생
160행 문제가 발생
161행 문제가 발생
163행 문제가 발생
173행 문제가 발생
175행 문제가 발생
178행 문제가 발생
179행 문제가 발생
180행 문제가 발생
181행 문제가 발생
184행 문제가 발생
186행 문제가 발생
187행 문제가 발생
189행 문제가 발생
190행 문제가 발생
193행 문제가 발생
194행 문제가 발생
195행 문제가 발생
196행 문제가 발생
199행 문제가 발생
200행 문제가 발생
204행 문제가 발생
208행 문제가 발생
209행 문제가 발생
211행 문제가 발생
217행 문제가 발생
226행 문제가 발생
228행 문제가 발생
230행 문제가 발생
231행 문제가 발생
233행 문제가 발생
234행 문제가 발생
235행 문제가 발생
242행 문제

C:\Users\default.DESKTOP-FVT3076\anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\default.DESKTOP-FVT3076\anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\default.DESKTOP-FVT3076\anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [257]:
# 크롤링 파일 저장

import os
from datetime import datetime

today = str(datetime.date(datetime.today()))
file = today + '_687행.csv'

if os.path.isfile(file):
    os.remove(file)
df.to_csv(file, encoding='utf-8')
# df.to_csv(today + '_test.csv', encoding='cp949')
print(file)

2021-09-23_687행.csv


In [ ]:
# %%cmd
# head C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json
# 안되니까 git bash로 확인
# 한줄 데이터라 byte 단위로 json 구조 확인
# head $ head data.json -c 1000

In [ ]:
# #데이터가 너무 커서 블록단위로 메모리에 올리는 ijson 사용
# import ijson
# import pandas

# with open('C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json','r',encoding='utf-8') as f:
#     obj = ijson.items(f, 'columns.item')
#     columns = list(obj)
#     print(columns[0])